In [ ]:
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
import glob

In [ ]:
#User Input Below
map_dir = r"C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\niftis_to_overlap"
file_pattern  = '*00*.nii'
analysis = "thresholded_rtoz_rios"

out_dir = r'C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\thresholded_maps'

In [ ]:
#preparing
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

glob_path = os.path.join(map_dir, file_pattern)
print('Path: \n', glob_path)
globbed = glob.glob(glob_path)
for i in range(0, len(globbed)):
    print(f'\n Nifti {i} found: \n', globbed[i])

In [ ]:
mask = nimds.get_img("mni_icbm152")
def topography(path, num_bands=5):
    img = image.load_img(path)
    mx = img.get_fdata()
    mx = np.nan_to_num(mx, nan=0, posinf=0, neginf=0)
    maximum = np.max(abs(mx))
    step = maximum/num_bands
    bands = [0.01]

    pos_tpg_mx = np.zeros(mx.shape)
    neg_tpg_mx = np.zeros(mx.shape)
    for i in range(0, num_bands+1):
        bands.append(np.round(step*(1+i), 2))
        pos_tpg_mx[mx>bands[i]] = bands[i]
        neg_tpg_mx[mx<-bands[i]] = -bands[i]
    tpg_mx = pos_tpg_mx + neg_tpg_mx
    print('Max is: ', np.max(abs(mx)))
    return tpg_mx, bands


topographic_matrix, bands = topography(globbed[0], 3)
print('Topographic bands: ', bands)
tpg_img = image.new_img_like(mask, topographic_matrix)
tpg_html = plotting.view_img(tpg_img, cut_coords=(0,0,0), title=(bands), black_bg=False, opacity=0.75, cmap='CMRmap')
tpg_html

In [ ]:
#Save Images
tpg_img.to_filename(os.path.join(out_dir, f'{analysis}'))
tpg_html.save_as_html(os.path.join(out_dir, f'{analysis}_.html'))